Import packages:

In [5]:
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'numpy'

Check pandas version (to doublecheck if loaded):

In [3]:
pd.__version__

NameError: name 'pd' is not defined

### Importing and inspecting data

Import data from CSV: `inventory.csv`

In [ ]:
df = pd.read_csv("inventory.csv") # df is the typical abbreviation for dataframe! Careful with the working directory!
df


,Substance,bottle_volume,remaining_amount,price
0,Acetone,500,120.0,8.5
1,Acetone,1000,840.0,14.0
2,Ethanol,1000,600.0,12.0
3,Ethanol,500,120.0,6.8
4,Ethanol,250,NaN,3.5
5,Sodium chloride,250,200.0,1.2
6,Hydrochloric acid,500,320.0,9.9
7,Acetic acid,100,NaN,4.6
8,Methanol,1000,910.0,11.5
9,Sodium hydroxide,500,240.0,7.2


In [5]:
pwd

'c:\\Users\\richa\\OneDrive\\Master\\Semester 2\\DSA 104\\DSA104_fork\\lectures\\session02\\material'

Get basic information about dataset (info, shape, size):

In [11]:
df.info()
df.size

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Substance         11 non-null     object 
 1   bottle_volume     11 non-null     int64  
 2   remaining_amount  9 non-null      float64
 3   price             11 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 480.0+ bytes


44

Summary statistics: describe!

In [12]:
df.describe()

,bottle_volume,remaining_amount,price
count,11.000000,9.000000,11.000000
mean,554.545455,398.888889,7.854545
std,316.586912,305.795865,3.855480
min,100.000000,120.000000,1.200000
25%,375.000000,200.000000,5.700000
50%,500.000000,240.000000,7.200000
75%,750.000000,600.000000,10.700000
max,1000.000000,910.000000,14.000000


### Indexing & selection
Columns can be called by simply selecting the column name from the dataframe, e.g. `df["name"]`. Multiple columns can be listed! This can also be used to create new columns.



In [16]:
df_test = df.copy()[["price", "bottle_volume"]]
df_test

,price,bottle_volume
0,8.5,500
1,14.0,1000
2,12.0,1000
3,6.8,500
4,3.5,250
5,1.2,250
6,9.9,500
7,4.6,100
8,11.5,1000
9,7.2,500


Entries can be localized by their coordinates using `loc` (explicit indices) and `iloc` (implicit indices). Can be used to change single values.

In [17]:
df.iloc[0,0]

'Acetone'

This can also be used for slicing! e.g. `df.iloc[startrow:endrow:interval, startcolumn:endcolumn:interval]` (endpoints not included! if one of the numbers not given: default = from start to end with interval 1, i.e. every entry).

In [ ]:
df_small = df.iloc[0:3:,::] # 3: means that 3 is not included
df_small

,Substance,bottle_volume,remaining_amount,price
0,Acetone,500,120.0,8.5
1,Acetone,1000,840.0,14.0
2,Ethanol,1000,600.0,12.0


Columns are accessed using the `columns` attribute, rows using the `index` attribute (can be used for renaming the whole axis).

In [ ]:
df_small.columns = ["A", "B", "C", "D"] #can also rename the name of the column to save time tipping
df_small

,A,B,C,D
0,Acetone,500,120.0,8.5
1,Acetone,1000,840.0,14.0
2,Ethanol,1000,600.0,12.0


Creating new columns, e.g. price per volume:

In [22]:
df["price_per_vol"] = df["price"]/df["bottle_volume"] # equivalent to:
# new_df = df.assign(price_per_vol = df["price"]/df["bottle_volume"]) 
df

,Substance,bottle_volume,remaining_amount,price,price_per_vol
0,Acetone,500,120.0,8.5,0.0170
1,Acetone,1000,840.0,14.0,0.0140
2,Ethanol,1000,600.0,12.0,0.0120
3,Ethanol,500,120.0,6.8,0.0136
4,Ethanol,250,NaN,3.5,0.0140
5,Sodium chloride,250,200.0,1.2,0.0048
6,Hydrochloric acid,500,320.0,9.9,0.0198
7,Acetic acid,100,NaN,4.6,0.0460
8,Methanol,1000,910.0,11.5,0.0115
9,Sodium hydroxide,500,240.0,7.2,0.0144


Selecting with certain condition: Boolean masks for selecting entries:

In [25]:
df.loc[df["Substance"] == "Ethanol"]

,Substance,bottle_volume,remaining_amount,price,price_per_vol
2,Ethanol,1000,600.0,12.0,0.0120
3,Ethanol,500,120.0,6.8,0.0136
4,Ethanol,250,NaN,3.5,0.0140


### Data cleaning & processing

To delete rows and columns, `drop` can be used (specifying the axis!):

In [29]:
# Drop the last column on a copy of teh original df
df_reduced = df.copy().drop(columns=["price_per_vol"], axis=1)
df_reduced

,Substance,bottle_volume,remaining_amount,price
0,Acetone,500,120.0,8.5
1,Acetone,1000,840.0,14.0
2,Ethanol,1000,600.0,12.0
3,Ethanol,500,120.0,6.8
4,Ethanol,250,NaN,3.5
5,Sodium chloride,250,200.0,1.2
6,Hydrochloric acid,500,320.0,9.9
7,Acetic acid,100,NaN,4.6
8,Methanol,1000,910.0,11.5
9,Sodium hydroxide,500,240.0,7.2


In [31]:
df_reduced2 = df.copy().drop(index=[0], axis=0)
df_reduced2

,Substance,bottle_volume,remaining_amount,price,price_per_vol
1,Acetone,1000,840.0,14.0,0.0140
2,Ethanol,1000,600.0,12.0,0.0120
3,Ethanol,500,120.0,6.8,0.0136
4,Ethanol,250,NaN,3.5,0.0140
5,Sodium chloride,250,200.0,1.2,0.0048
6,Hydrochloric acid,500,320.0,9.9,0.0198
7,Acetic acid,100,NaN,4.6,0.0460
8,Methanol,1000,910.0,11.5,0.0115
9,Sodium hydroxide,500,240.0,7.2,0.0144
10,Sodium hydroxide,500,240.0,7.2,0.0144


Most important use cases: handle missing values and duplicates.

Detect missing values: `isna()` - best used in combination with `sum()` or `any()`. Can be used on df or columns.

In [32]:
df.isna().any()

Substance           False
bottle_volume       False
remaining_amount     True
price               False
price_per_vol       False
dtype: bool

If desired, rows with missing values can be dropped with `dropna()` (e.g. if crucial value). Good practice: Documentation! And keep original df as copy!

In [33]:
df2 = df.copy().dropna()
df2

,Substance,bottle_volume,remaining_amount,price,price_per_vol
0,Acetone,500,120.0,8.5,0.0170
1,Acetone,1000,840.0,14.0,0.0140
2,Ethanol,1000,600.0,12.0,0.0120
3,Ethanol,500,120.0,6.8,0.0136
5,Sodium chloride,250,200.0,1.2,0.0048
6,Hydrochloric acid,500,320.0,9.9,0.0198
8,Methanol,1000,910.0,11.5,0.0115
9,Sodium hydroxide,500,240.0,7.2,0.0144
10,Sodium hydroxide,500,240.0,7.2,0.0144


Duplicates can be detected by `duplicated()`. Duplicates can be dropped using `drop_duplicates()`. Good practice: Documentation and working on copies!

In [34]:
df.duplicated().sum()

1

Both cases, duplicates and missing values, can be treated with other options (see documentation)! Or completely different approaches may be valid, such as filling or flagging.

Types (info!) can be redefined using `astype`:

In [35]:
print(df.info())
df["bottle_volume"] = df["bottle_volume"].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Substance         11 non-null     object 
 1   bottle_volume     11 non-null     int64  
 2   remaining_amount  9 non-null      float64
 3   price             11 non-null     float64
 4   price_per_vol     11 non-null     float64
dtypes: float64(3), int64(1), object(1)
memory usage: 568.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Substance         11 non-null     object 
 1   bottle_volume     11 non-null     float64
 2   remaining_amount  9 non-null      float64
 3   price             11 non-null     float64
 4   price_per_vol     11 non-null     float64
dtypes: float64(4), object(1)
memory usage: 568.0+ bytes


### Grouping & aggregation

`groupby` is an important example of a split-apply-combine approach: first, entries in a specified column will be grouped together, then a function is applied (e.g. an aggregate such as mean, std.dev., sum) to all numerical values in each group, then the groups are recombined to give another dataframe.

Single columns can be selected from the groups to which the aggregates will be applied. Use `agg()` to mix different aggregates for different columns (refer to documentation).

In [36]:
df.groupby("Substance").mean()

,bottle_volume,remaining_amount,price,price_per_vol
Substance,,,,
Acetic acid,100.000000,NaN,4.600000,0.0460
Acetone,750.000000,480.0,11.250000,0.0155
Ethanol,583.333333,360.0,7.433333,0.0132
Hydrochloric acid,500.000000,320.0,9.900000,0.0198
Methanol,1000.000000,910.0,11.500000,0.0115
Sodium chloride,250.000000,200.0,1.200000,0.0048
Sodium hydroxide,500.000000,240.0,7.200000,0.0144


### Merging and joining datasets

`pd.merge` and `pd.concat` are very important functions to combine different Dataframes (or Series).

First load as `df2` the CSV file `chemicals.csv`:

In [37]:
df2 = pd.read_csv("chemicals.csv")
df2

,Substance,Chem_ID,SMILES
0,Acetone,67-64-1,CC(=O)C
1,Ethanol,64-17-5,CCO
2,Sodium chloride,7647-14-5,[Na+].[Cl-]
3,Hydrochloric acid,7647-01-0,Cl
4,Acetic acid,64-19-7,CC(=O)O
5,Methanol,67-56-1,CO
6,Sodium hydroxide,1310-73-2,[Na+].[OH-]
7,Isopropanol,67-63-0,CC(C)O
8,Toluene,108-88-3,Cc1ccccc1


`pd.concat` mainly appends the dataframes, per default as new rows (can be specified with axis!). `join=` defines if everything is kept ("outer") or only the entries that are existing in both ("inner"). `ignore_index=True` will replace the original indices.

In [38]:
df_appended = pd.concat((df, df2), join="outer", ignore_index=True)
df_appended

,Substance,bottle_volume,remaining_amount,price,price_per_vol,Chem_ID,SMILES
0,Acetone,500.0,120.0,8.5,0.0170,NaN,NaN
1,Acetone,1000.0,840.0,14.0,0.0140,NaN,NaN
2,Ethanol,1000.0,600.0,12.0,0.0120,NaN,NaN
3,Ethanol,500.0,120.0,6.8,0.0136,NaN,NaN
4,Ethanol,250.0,NaN,3.5,0.0140,NaN,NaN
5,Sodium chloride,250.0,200.0,1.2,0.0048,NaN,NaN
6,Hydrochloric acid,500.0,320.0,9.9,0.0198,NaN,NaN
7,Acetic acid,100.0,NaN,4.6,0.0460,NaN,NaN
8,Methanol,1000.0,910.0,11.5,0.0115,NaN,NaN
9,Sodium hydroxide,500.0,240.0,7.2,0.0144,NaN,NaN


`pd.merge` fuses two dataframes together (specified with `left=` and `right=`) `on` a specified common key, `how` sets the merge type (similar as to the join type with `concat`, but with more options). Merges can be also done on different keys (`left_on=`, `right_on=`). `suffixes` can fix clashes when more columns bear the same name in the two dataframes (see documentaiton).

In [39]:
df_merged = pd.merge(left=df, right=df2, on="Substance", how="outer")
df_merged

,Substance,bottle_volume,remaining_amount,price,price_per_vol,Chem_ID,SMILES
0,Acetic acid,100.0,NaN,4.6,0.0460,64-19-7,CC(=O)O
1,Acetone,500.0,120.0,8.5,0.0170,67-64-1,CC(=O)C
2,Acetone,1000.0,840.0,14.0,0.0140,67-64-1,CC(=O)C
3,Ethanol,1000.0,600.0,12.0,0.0120,64-17-5,CCO
4,Ethanol,500.0,120.0,6.8,0.0136,64-17-5,CCO
5,Ethanol,250.0,NaN,3.5,0.0140,64-17-5,CCO
6,Hydrochloric acid,500.0,320.0,9.9,0.0198,7647-01-0,Cl
7,Isopropanol,NaN,NaN,NaN,NaN,67-63-0,CC(C)O
8,Methanol,1000.0,910.0,11.5,0.0115,67-56-1,CO
9,Sodium chloride,250.0,200.0,1.2,0.0048,7647-14-5,[Na+].[Cl-]


### Reshaping data

`melt` transforms a "wide table" into a long one ("Take many columns and stack them into rows."). Often this is done for visualisation. The basic syntax is: `pd.melt(df, id_vars=..., value_vars=..., var_name=..., value_name=...)`, where...

...`id_vars`: columns to keep as identifiers (stay the same)
...`value_vars`: columns that will be unpivoted (melted into rows)
...`var_name`: name of the new column that will hold the former column names
...`value_name`: name of the new column that will hold the values

`pivot` is the reverse operation, `pivot_table` works in a similar way - refer to documentation.

In [40]:
# not the most meaningful example... (better one see dht_data)
print(df)
df_molten = df.melt(
    id_vars="Substance", 
    value_vars=["bottle_volume", "remaining_amount", "price", "price_per_vol"],
    var_name="container_property",
    value_name="value"
    )
df_molten

            Substance  bottle_volume  remaining_amount  price  price_per_vol
0             Acetone          500.0             120.0    8.5         0.0170
1             Acetone         1000.0             840.0   14.0         0.0140
2             Ethanol         1000.0             600.0   12.0         0.0120
3             Ethanol          500.0             120.0    6.8         0.0136
4             Ethanol          250.0               NaN    3.5         0.0140
5     Sodium chloride          250.0             200.0    1.2         0.0048
6   Hydrochloric acid          500.0             320.0    9.9         0.0198
7         Acetic acid          100.0               NaN    4.6         0.0460
8            Methanol         1000.0             910.0   11.5         0.0115
9    Sodium hydroxide          500.0             240.0    7.2         0.0144
10   Sodium hydroxide          500.0             240.0    7.2         0.0144


,Substance,container_property,value
0,Acetone,bottle_volume,500.0000
1,Acetone,bottle_volume,1000.0000
2,Ethanol,bottle_volume,1000.0000
3,Ethanol,bottle_volume,500.0000
4,Ethanol,bottle_volume,250.0000
5,Sodium chloride,bottle_volume,250.0000
6,Hydrochloric acid,bottle_volume,500.0000
7,Acetic acid,bottle_volume,100.0000
8,Methanol,bottle_volume,1000.0000
9,Sodium hydroxide,bottle_volume,500.0000
